# one-off preparation of the database

In [4]:
!lamin init --storage ~/scprint2 --schema bionty

✅ saved: User(uid='a3keNAVX', handle='jkobject', updated_at=2023-12-08 12:31:42 UTC)
✅ saved: Storage(uid='tX3REfAQ', root='/home/ml4ig1/scprint2', type='local', updated_at=2023-12-08 12:31:42 UTC, created_by_id=1)
💡 loaded instance: jkobject/scprint2
💡 did not register local instance on hub


In [1]:
import bionty as bt
import lamindb as ln
import lnschema_bionty as lb
import pandas as pd
from scprint import data_utils

%load_ext autoreload
%autoreload 2

💡 lamindb instance: jkobject/scprint


In [2]:
lb.settings.organism = "human"

## prepare lamin database

In [4]:
cx_dataset = ln.Collection.using("laminlabs/cellxgene").one()
cx_dataset

Dataset(uid='OirHTWDrudY2TYltvIX1', name='cellxgene-census', version='2023-07-25', hash='pEJ9uvIeTLvHkZW2TBT5', visibility=1, updated_at=2023-11-28 21:46:40 UTC, transform_id=11, run_id=16, created_by_id=1)

In [ ]:
mydataset = data_utils.load_dataset_local(lb, cx_dataset, "~/scprint/", name="cellxgene-local", description="the full cellxgene database", only=(229,800), use_cache=True)

In [5]:
files = ln.Artifact.filter()
#loaded = ! ls ~/scprint/cell-census/2023-07-25/h5ads/

In [83]:
name="initial dataset"
description="directly from cellxgene"
dataset = ln.Collection(mfiles, name=name, description=description)
dataset.save()

❗ no run & transform get linked, consider passing a `run` or calling ln.track()


❗ returning existing dataset with same hash: Dataset(uid='JPTWFUdspvbX8OxdO4LL', name='initial dataset', description='directly from cellxgene', hash='luh-kLdfruzsj5K3lbWG', visibility=1, updated_at=2023-12-07 10:18:58 UTC, created_by_id=1)


In [20]:
len(mfiles)

506

## load some known ontology names

In [9]:
#you can also load it back
mydataset = ln.Collection.filter(name="cellxgene-local").one()

In [1]:
import cellxgene_census

census = cellxgene_census.open_soma(census_version = "latest")
val_to_get = ['self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id']
df = census["census_data"]["homo_sapiens"].obs.read(column_names=val_to_get, value_filter="is_primary_data == True").concat().to_pandas()
df2 = census["census_data"]["mus_musculus"].obs.read(column_names=val_to_get, value_filter="is_primary_data == True").concat().to_pandas()
df.shape

(39055600, 6)

In [17]:
records = lb.Organism.from_values(["NCBITaxon:10090"], field=lb.Organism.ontology_id)
ln.save([records[0]])
lb.Organism(name="unknown", ontology_id="unknown").save()

❗ ambiguous validation in Bionty for 1 record: 'NCBITaxon:10090'


In [7]:
lb.BiontySource.filter(entity="Gene", organism='mouse').first()

BiontySource(uid='VTEw', entity='Gene', organism='mouse', currently_used=True, source='ensembl', source_name='Ensembl', version='release-110', url='s3://bionty-assets/df_mouse__ensembl__release-110__Gene.parquet', md5='fa4ce130f2929aefd7ac3bc8eaf0c4de', source_website='https://www.ensembl.org', updated_at=2023-11-22 09:47:07 UTC, created_by_id=1)

In [17]:
set(lb.Gene.filter().df()['organism_id'])

{1, 2, 4, 5}

In [81]:
data_utils.populate_my_ontology(lb=lb,
    #organisms=["NCBITaxon:10090",],# "NCBITaxon:9606"],
    sex=["PATO:0000384", "PATO:0000383"],
    ethnicities=df['self_reported_ethnicity_ontology_term_id'].unique().tolist(),
    assays=list(set(df['assay_ontology_term_id'].unique()).union(df2['assay_ontology_term_id'].unique())) + ['EFO:0010961'],
    tissues=list(set(df['tissue_ontology_term_id'].unique()).union(df2['tissue_ontology_term_id'].unique())),
    # we load all possible diseases. makes it easier
    #diseases=list(set(df['disease_ontology_term_id'].unique()).union(df2['disease_ontology_term_id'].unique())),
    # we load all possible cell types too.
    dev_stages=list(df['development_stage_ontology_term_id'].unique()),)

> /home/ml4ig1/Documents code/scPRINT/scprint/dataset/utils.py(204)populate_my_ontology()
    202 
    203     pdb.set_trace()
--> 204     names = bt.CellType().df().index if not celltypes else celltypes
    205     records = lb.CellType.from_values(names, field=lb.CellType.ontology_id)
    206     ln.save(records)

❗ now recursing through parents: this only happens once, but is much slower than bulk saving
❗ did not create Ethnicity record for 1 non-validated ontology_id: 'multiethnic'
❗ now recursing through parents: this only happens once, but is much slower than bulk saving
❗ did not create DevelopmentalStage record for 1 non-validated ontology_id: 'unknown'
❗ now recursing through parents: this only happens once, but is much slower than bulk saving
❗ now recursing through parents: this only happens once, but is much slower than bulk saving
❗ records with similar names exist! did you mean to load one of them?


,uid,synonyms,score
name,,,
Noonan syndrome,PR8Zk54a,"Noonan's syndrome|Turner's phenotype, karyotyp...",90.0
PLG-related hereditary angioedema with normal C1inh,nWJslCey,PLG-related HAE with normal C1 inhibitor,90.0
Pacinian tumor,7oJdjev9,Pacinian neurofibroma|Pacinian tumor (morpholo...,90.0
abnormal pupillary function,OcIbPVkb,abnormal pupillary function (disease)|abnormal...,90.0
abnormal threshold of rods,FLtCc0CF,abnormal dark adaptation curve,90.0
acinar cell carcinoma,5eieG3Hz,acinar cell adenocarcinoma|acinar carcinoma|ac...,90.0
acute myeloid leukemia with 11q23 abnormalities,3esmrKXF,acute myeloid Leukemia with t(9;11)(p21.3;q23....,90.0
adenocarcinoma,O8eOaxZg,"adenocarcinoma, malignant|adenocarcinomas|aden...",90.0
adenomatoid tumor,pkFcp0xT,adenomatoid tumor|adenomatoid tumor (morpholog...,90.0


❗ records with similar names exist! did you mean to load one of them?


,uid,synonyms,score
name,,,
cancer of unknown primary site,wBFcNNtH,,90.0
chronic intervillositis of unknown etiology,d10sK6bb,CIUE,90.0
monoclonal gammopathy of uncertain significance,RWETMdHi,"monoclonal gammopathy, benign|benign monoclona...",90.0
mucinous adenocarcinoma,uCV8mi51,colloid adenocarcinoma|mucin-producing adenoca...,90.0
sporadic adult-onset ataxia of unknown etiology,ciICnEgz,SAOA|idiopathic late-onset cerebellar ataxia,90.0
unknown leukodystrophy,im1dGXZx,,90.0


❗ did not create Gene records for 56941 non-validated ensembl_gene_ids: 'ENSMUSG00000000001', 'ENSMUSG00000000003', 'ENSMUSG00000000028', 'ENSMUSG00000000031', 'ENSMUSG00000000037', 'ENSMUSG00000000049', 'ENSMUSG00000000056', 'ENSMUSG00000000058', 'ENSMUSG00000000078', 'ENSMUSG00000000085', 'ENSMUSG00000000088', 'ENSMUSG00000000093', 'ENSMUSG00000000094', 'ENSMUSG00000000103', 'ENSMUSG00000000120', 'ENSMUSG00000000125', 'ENSMUSG00000000126', 'ENSMUSG00000000127', 'ENSMUSG00000000131', 'ENSMUSG00000000134', ...


In [18]:

bionty_source_ds_mouse = lb.BiontySource.filter(entity="DevelopmentalStage", organism="mouse").one()
records = lb.DevelopmentalStage.from_values(df2['development_stage_ontology_term_id'].unique().tolist(), field=lb.DevelopmentalStage.ontology_id, bionty_source=bionty_source_ds_mouse)
ln.save(records)

NameError: name 'df2' is not defined

In [19]:
assay = ['EFO:0010961']

In [20]:
records = lb.ExperimentalFactor.from_values(assay, field=lb.ExperimentalFactor.ontology_id)
ln.save(records)

## add some missing ontology names

In [5]:
from scprint.dataset.utils import get_ancestry_mapping

In [ ]:
additional_tissues = {
    "UBERON:0037144": "wall of heart",
    "UBERON:0003929": "digestive tract epithelium",
    "UBERON:0002020": "gray matter",
    "UBERON:0000200": "gyrus",
    "UBERON:0000101": "lobe of lung",
    "UBERON:0001981": "blood vessel",
    "UBERON:0001474": "bone element",
}

additional_diseases = {
    "MONDO:0001106": "kidney failure",
    "MONDO:0021166": "inflammatory disease",
    "MONDO:0004992": "cancer",
    "MONDO:0004994": "cardiomyopathy",
    "MONDO:0700065": "trisomy",
    "MONDO:0021042": "glioma",
    "MONDO:0005265": "inflammatory bowel disease",
    "MONDO:0005550": "infectious disease",
    "MONDO:0005059": "leukemia",
}

additional_assays = {
    "EFO:0010184": "Smart-like",
}


Did it using the code below to figure out things we might want to add etc..

In [ ]:
mapping, anc, leafs = get_ancestry_mapping(df['tissue_ontology_term_id'].unique(), lb.Tissue.filter().df(include=["parents__ontology_id"]).set_index("ontology_id"))
# getting only the leaves for which we don't have a parent
leafs = list(leafs - set.union(*[mapping[val] for val in mapping.keys()]))

In [ ]:
lb.Tissue.search(list(leafs)[108], field="ontology_id",return_queryset=True).first().view_parents()

## Preprocessing

In [5]:
import numpy as np
import scanpy as sc
from scprint.dataset.preprocess import Preprocessor


In [ ]:
adata

In [21]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype(str)

In [22]:
adata.obs.loc[loc, 'tissue_ontology_term_id'] = adata.obs.loc[loc, 'tissue_ontology_term_id'].str.replace(" (cell culture)", "")

In [17]:
adata.obs['cell_culture'] = False
# if cell_type contains the word "(cell culture)" then it is a cell culture and we mark it as so and remove this from the cell type
loc = adata.obs['tissue_ontology_term_id'].str.contains("(cell culture)")
loc.sum()

77650

In [23]:
def additional_preprocess(adata):
    adata.obs = adata.obs.replace({'self_reported_ethnicity_ontology_term_id':{
        'multiethnic':'unknown',
        'American':'unknown',
        'Jewish Israeli': 'unknown',
        'na':'unknown',
    }}) #multi ethnic will have to get renamed
    adata.obs['cell_culture'] = False
    # if cell_type contains the word "(cell culture)" then it is a cell culture and we mark it as so and remove this from the cell type
    loc = adata.obs['cell_type_ontology_term_id'].str.contains("(cell culture)")
    if loc.sum()>0:
        adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype(str)
        adata.obs.loc[loc, 'cell_culture'] = True
        adata.obs.loc[loc, 'cell_type_ontology_term_id'] = adata.obs.loc[loc, 'cell_type_ontology_term_id'].str.replace(" (cell culture)", "")
    loc = adata.obs['tissue_ontology_term_id'].str.contains("(cell culture)")
    if loc.sum()>0:
        adata.obs.loc[loc, 'cell_culture'] = True
        adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype(str)
        adata.obs.loc[loc, 'tissue_ontology_term_id'] = adata.obs.loc[loc, 'tissue_ontology_term_id'].str.replace(" (cell culture)", "")
    loc = adata.obs['tissue_ontology_term_id'].str.contains("CL:")
    if loc.sum()>0:
        adata.obs.loc[loc, 'tissue_ontology_term_id'] = "unknown"
    return adata

def additional_postprocess(adata):
    # define the "up to" 10 neighbors for each cells and add to obs
    # compute neighbors
    # need to be connectivities and same labels [cell type, assay, dataset, disease]
    # define the "neighbor" up to 10(N) cells and add to obs
    # define the "next time point" up to 5(M) cells and add to obs  # step 1: filter genes
    sc.tl.diffmap(adata)
    # create a meta group
    adata.obs['dpt_group'] = adata.obs['leiden_1'].astype(str) + "_" + adata.obs['disease_ontology_term_id'].astype(str) + "_" + adata.obs['cell_type_ontology_term_id'].astype(str) + "_" + adata.obs['tissue_ontology_term_id'].astype(str) #+ "_" + adata.obs['dataset_id'].astype(str)

    # if group is too small
    okgroup = [i for i, j in adata.obs['dpt_group'].value_counts().items() if j>=10]
    not_okgroup = [i for i, j in adata.obs['dpt_group'].value_counts().items() if j<3]
    # set the group to empty
    adata.obs.loc[adata.obs['dpt_group'].isin(not_okgroup), 'dpt_group'] = ''
    adata.obs['heat_diff'] = np.nan
    # for each group
    for val in set(okgroup):
        if val == '':
            continue
        # get the best root cell
        eq = adata.obs.dpt_group==val
        loc = np.where(eq)[0]

        root_ixs = loc[adata.obsm["X_diffmap"][eq, 0].argmin()]
        adata.uns["iroot"] = root_ixs
        # compute the diffusion pseudo time from it
        sc.tl.dpt(adata)
        adata.obs.loc[eq, 'heat_diff'] = adata.obs.loc[eq, 'dpt_pseudotime']
        adata.obs.drop(columns=['dpt_pseudotime'], inplace=True)

    #sort so that the next time points are aligned for all groups
    adata = adata[adata.obs.sort_values(['dpt_group','heat_diff']).index]
    #to query N next time points we just get the N elements below and check they are in the group
    # to query the N nearest neighbors we just get the N elements above and N below and check they are in the group
    return adata

do_preprocess = Preprocessor(lb, additional_postprocess=additional_postprocess, additional_preprocess=additional_preprocess, cache=True)


In [ ]:
# https://scikit-cuda.readthedocs.io/en/latest/generated/skcuda.linalg.PCA.html

In [24]:
import warnings

warnings.filterwarnings("ignore", category=ResourceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

preprocessed_dataset = do_preprocess(cx_dataset, start_at=8)

❗ no run & transform get linked, consider passing a `run` or calling ln.track()


0
File(uid='tQaZHQ9Lyy5AE2n5RC2x', key='cell-census/2023-07-25/h5ads/030faa69-ff79-4d85-8630-7c874a114c19.h5ad', suffix='.h5ad', accessor='AnnData', description='Single-cell longitudinal analysis of SARS-CoV-2 infection in human bronchial epithelial cells', size=1090725687, hash='HK6jRnvyiZ4x7ln7InNj-g-131', hash_type='md5-n', visibility=1, key_is_virtual=False, updated_at=2023-11-28 22:44:31 UTC, storage_id=2, transform_id=11, run_id=16, created_by_id=1)
AnnData object with n_obs × n_vars = 77650 × 24482
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'pmito', 'n_genes', 'Condition', 'louvain', 'Infected', 'Cell type', 'assay_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_onto

IndexError: list index out of range

In [3]:
#we have processed that many files
len(ln.Artifact.filter(version='2', description='preprocessed by scprint'))

13

In [3]:
# I need to remake the dataset as it failed for some files and I had to restart at position 11
name="preprocessed dataset"
description="preprocessed dataset using scprint"
dataset = ln.Collection(ln.Artifact.filter(version='2', description='preprocessed by scprint'), name=name, description=description)
dataset.save()
dataset.files.count()

❗ no run & transform get linked, consider passing a `run` or calling ln.track()


## Gene embeddings

In [19]:
from scprint.dataloader.embedder import embed

In [3]:
genedf = lb.Gene.filter(organism).df()

In [16]:
lb.Organism.filter(ontology_id="NCBITaxon:9606").one().scientific_name

'homo_sapiens'

In [12]:
organism = lb.Organism.filter(ontology_id="NCBITaxon:9606").one()

In [11]:
organism

'human'

In [4]:
genedf

,uid,symbol,stable_id,ensembl_gene_id,ncbi_gene_ids,biotype,description,synonyms,organism_id,bionty_source_id,updated_at,created_by_id
id,,,,,,,,,,,,
53102,a6Xjf000WNY3,LINC01772,None,ENSG00000226029,107984921,lncRNA,long intergenic non-protein coding RNA 1772 [S...,ENSG00000226029,2,9,2023-11-22 13:16:32.368761+00:00,1
53103,zR8psPjThixh,LINC01772,None,ENSG00000291388,107984921,lncRNA,long intergenic non-protein coding RNA 1772 [S...,ENSG00000226029,2,9,2023-11-22 13:16:32.368797+00:00,1
53104,mRzSiS7BgReH,TSPAN6,None,ENSG00000000003,7105,protein_coding,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858],T245|TSPAN-6|TM4SF6,2,9,2023-11-22 13:16:56.402584+00:00,1
53105,2wlFrfmHXEzi,TNMD,None,ENSG00000000005,64102,protein_coding,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757],TEM|CHM1L|BRICD4|MYODULIN|TENDIN,2,9,2023-11-22 13:16:56.402622+00:00,1
53106,NQ1m5CzgdgDN,DPM1,None,ENSG00000000419,8813,protein_coding,dolichyl-phosphate mannosyltransferase subunit...,MPDS|CDGIE,2,9,2023-11-22 13:16:56.402654+00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...
180154,hk0qmgaJts9K,Gm56371,None,ENSMUSG00002076988,,rRNA,"predicted gene, 56371 [Source:MGI Symbol;Acc:M...",,1,11,2023-11-28 14:29:52.039397+00:00,1
180155,jRNdAmsstLGL,Gm23510,None,ENSMUSG00002076989,115490486,snRNA,"predicted gene, 23510 [Source:MGI Symbol;Acc:M...",,1,11,2023-11-28 14:29:52.039424+00:00,1
180156,40YbTKFTR0L7,Gm22711,None,ENSMUSG00002076990,,snoRNA,"predicted gene, 22711 [Source:MGI Symbol;Acc:M...",,1,11,2023-11-28 14:29:52.039452+00:00,1


In [ ]:
genedf = lb.Gene.filter().df()

embeddings = embed(genedf=genedf,
    organism="homo_sapiens",
    cache=True,
    fasta_path="/tmp/data/fasta/",
    embedding_size=128,)

In [ ]:
genedf = pd.concat([genedf, embeddings], axis=1, join='inner')
genesdf.to_parquet('../../data/temp/genesdf.parquet')

## data loader

In [20]:
from torch.utils.data import WeightedRandomSampler

In [28]:
1 in list(WeightedRandomSampler(np.array([0,0,0,0,1,0.1,0.3,4,4])/100000, num_samples=1000, replacement=True))

False

In [ ]:
dataset = ln.Collection.filter(name='preprocessed dataset').one()
genesdf = pd.read_parquet('../../data/temp/genesdf.parquet')

In [ ]:
# TODO: finish the annotation things DONE
# TODO: make the dataloader for the model
# TODO: add gene length to the preprocessor if needed

In [41]:
from scprint.dataset import mapped

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

  %reload_ext autoreload


In [134]:
mapped_dataset = mapped.mapped(dataset, label_keys=all_labels, encode_labels=labels_weighted_sampling)

❗ no run & transform get linked, consider passing a `run` or calling ln.track()


In [53]:
weights = mapped_dataset.get_label_weights(labels_weighted_sampling)

In [35]:
labels_weighted_sampling = [
    'self_reported_ethnicity_ontology_term_id',
    'assay_ontology_term_id',
    'development_stage_ontology_term_id',
    'disease_ontology_term_id',
    'cell_type_ontology_term_id',
    'tissue_ontology_term_id',
    'sex_ontology_term_id',
]

all_labels = [
    'self_reported_ethnicity_ontology_term_id',
    'assay_ontology_term_id',
    'development_stage_ontology_term_id',
    'disease_ontology_term_id',
    'cell_type_ontology_term_id',
    'tissue_ontology_term_id',
    'sex_ontology_term_id',
    #'dataset_id',
    #'cell_culture',
    "dpt_group",
    "heat_diff",
    "nnz",
]

In [37]:
len(mapped_dataset)

364376

In [ ]:
# get the N samples DONE
# get the unseen info DONE
# get the I most expressed genes, add randomly some unexpressed genes that are not unseen
# map the genes to the embeddings DONE
# create positions from counts (function)
# add to embeddings
# create / learn special tokens embeddings. <batch> <class> <library_size> <diffpseudotime>
# other version, one token per class type <self_reported_ethnicity_ontology_term_id> <assay_ontology_term_id> <development_stage_ontology_term_id> <disease_ontology_term_id> <cell_type_ontology_term_id> <tissue_ontology_term_id> <sex_ontology_term_id>
# define positions for these tokens
# get all annotations and parental relations and convert to encoding
# get the N nearest neighbors if any
# get the 3 next time point if any
    # do the same for the 3 next time points